In [ ]:
import sys
sys.path.append("../")

from nlrl.utils import write_jsonl, read_jsonl

Merging state dataset, and split it into train and test set

Get test set 3000

In [ ]:
import os
root_dir = './rollout_collection_5x5'
data_list = os.listdir(root_dir)

final_data_list = []
for path in data_list:
    data_path = os.path.join(root_dir, path, 'replay_buffer.jsonl')
    final_data_list.append(data_path)

merged_data = []
for path in final_data_list:
    merged_data.extend(read_jsonl(path))

full_states = []
for d in merged_data:
    full_states.extend(d['serializad_state'])
print(len(full_states))
full_states = set(full_states)
print(len(full_states))
full_states = list(filter(lambda x: x != 'Terminal State.', full_states))

import numpy as np
np.random.seed(42)
evaluation = np.random.choice(full_states, 3000, replace=False)
evaluation_data = [{'serializad_state': e} for e in evaluation]

save_dir = './rollout_collection_5x5_processed/evaluation_3000'
write_jsonl(evaluation_data, os.path.join(save_dir, 'merged_initial_state.jsonl'), overwrite=True)

Get train set by removing 3000 test states

In [ ]:
import os
root_dir = './rollout_collection_5x5'
data_list = os.listdir(root_dir)
evaluation = read_jsonl('./rollout_collection_5x5_processed/evaluation_3000/merged_initial_state.jsonl')
evaluation = [e['serializad_state'] for e in evaluation]

final_data_list = []
for path in data_list:
    data_path = os.path.join(root_dir, path, 'replay_buffer.jsonl')
    final_data_list.append(data_path)

merged_data = []
for path in final_data_list:
    merged_data.extend(read_jsonl(path))

full_states = []
for d in merged_data:
    full_states.extend(d['serializad_state'])
print(len(full_states))
full_states = set(full_states)
print(len(full_states))
evaluation_ref = set(evaluation)
full_states = [f for f in full_states if not f in evaluation_ref]
print(len(full_states))
training_data = [{'serializad_state': e} for e in full_states]

save_dir = './rollout_collection_5x5_processed/train_without_eval_3000'

config = [{"comment": "This is the merged deduplicated state data without 3000 held-out evaluation set"}]
write_jsonl(config, os.path.join(save_dir, 'config.jsonl'), overwrite=True)
write_jsonl(training_data, os.path.join(save_dir, 'merged_initial_state.jsonl'), overwrite=True)

Subsample 1/40 dataset, around 10k unique states

In [ ]:
import os
root_dir = './rollout_collection_5x5'
save_dir = './rollout_collection_5x5_processed/sample_1_40'

train_set = read_jsonl('./rollout_collection_5x5_processed/train_without_eval_3000/merged_initial_state.jsonl')

import numpy as np
seed = 42
ratio = 0.025
np.random.seed(seed)
merged_data = np.random.choice(train_set, int(ratio * len(train_set)), replace=False)
print(len(merged_data))

config = [{"seed": seed, "ratio": ratio}]
write_jsonl(config, os.path.join(save_dir, 'config.jsonl'), overwrite=True)
write_jsonl(merged_data, os.path.join(save_dir, 'merged_initial_state.jsonl'), overwrite=True)